In [4]:
import pandas as pd
import numpy as np
import os

In [5]:
RAW_DATA_PATH = "data/raw/grocery_inventory_sales.csv"
PROCESSED_DATA_PATH = "data/processed/cleaned_inventory_sales.csv"

In [6]:
df = pd.read_csv(RAW_DATA_PATH)
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,6/29/2024,9/19/2024,48 Del Sol Trail,32,19,Discontinued
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11-01-2024,5/29/2024,05-08-2024,36 3rd Place,85,1,Discontinued
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,08-03-2024,06-10-2024,9/22/2024,3296 Walton Court,31,34,Backordered
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12-08-2024,2/19/2025,4/17/2024,3 Westerfield Crossing,95,99,Active
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,07-03-2024,10-11-2024,10-05-2024,15068 Scoville Court,62,25,Backordered


In [7]:
df.shape

(990, 16)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Product_ID               990 non-null    object
 1   Product_Name             990 non-null    object
 2   Catagory                 989 non-null    object
 3   Supplier_ID              990 non-null    object
 4   Supplier_Name            990 non-null    object
 5   Stock_Quantity           990 non-null    int64 
 6   Reorder_Level            990 non-null    int64 
 7   Reorder_Quantity         990 non-null    int64 
 8   Unit_Price               990 non-null    object
 9   Date_Received            990 non-null    object
 10  Last_Order_Date          990 non-null    object
 11  Expiration_Date          990 non-null    object
 12  Warehouse_Location       990 non-null    object
 13  Sales_Volume             990 non-null    int64 
 14  Inventory_Turnover_Rate  990 non-null    i

In [9]:
df.describe()

,Stock_Quantity,Reorder_Level,Reorder_Quantity,Sales_Volume,Inventory_Turnover_Rate
count,990.000000,990.000000,990.000000,990.000000,990.000000
mean,55.609091,51.215152,51.913131,58.925253,50.150505
std,26.300775,29.095241,29.521059,23.002318,28.798954
min,10.000000,1.000000,1.000000,20.000000,1.000000
25%,33.000000,25.250000,25.000000,39.000000,25.000000
50%,56.000000,53.000000,54.000000,58.000000,50.000000
75%,79.000000,77.000000,77.000000,78.000000,74.750000
max,100.000000,100.000000,100.000000,100.000000,100.000000


No of Rows: 990, No of Cols: 16, No of Numeric Vals: 5, No of Categorical Vals: 11

In [10]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

## 1.6 Handle Missing Values

In [11]:
df.isnull().sum()

product_id                 0
product_name               0
catagory                   1
supplier_id                0
supplier_name              0
stock_quantity             0
reorder_level              0
reorder_quantity           0
unit_price                 0
date_received              0
last_order_date            0
expiration_date            0
warehouse_location         0
sales_volume               0
inventory_turnover_rate    0
status                     0
dtype: int64

In [12]:
# Numeric columns → median
num_cols = df.select_dtypes(include=np.number).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical columns → mode
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

## 1.7 Remove Duplicates

In [13]:
df = df.drop_duplicates()

## 1.8 Business Column Mapping
Business Interpretation

sales → Demand

inventory → Supply

unit_price → Revenue impact

category → Segmentation

## 1.9 Data Type Fixing

In [14]:
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [15]:
df["unit_price"] = (
    df["unit_price"]
    .replace(r"[\$,]", "", regex=True)
    .astype(float)
)

In [16]:
date_cols = [
    "date_received",
    "last_order_date",
    "expiration_date"
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   product_id               990 non-null    object        
 1   product_name             990 non-null    object        
 2   catagory                 990 non-null    object        
 3   supplier_id              990 non-null    object        
 4   supplier_name            990 non-null    object        
 5   stock_quantity           990 non-null    int64         
 6   reorder_level            990 non-null    int64         
 7   reorder_quantity         990 non-null    int64         
 8   unit_price               990 non-null    float64       
 9   date_received            600 non-null    datetime64[ns]
 10  last_order_date          595 non-null    datetime64[ns]
 11  expiration_date          630 non-null    datetime64[ns]
 12  warehouse_location       990 non-nul

In [18]:
os.makedirs("../data/processed", exist_ok=True)
df.to_csv(PROCESSED_DATA_PATH, index=False)